In [10]:
import streamlit as st
import pandas as pd
from openai import OpenAI
import os
import json
import time
from datetime import datetime
from dotenv import load_dotenv
from PIL import Image

In [19]:
if load_dotenv():
    print(".env file loaded successfully")
else:
    print("Failed to load .env file")

Failed to load .env file


In [23]:
#load_dotenv('.env')
test_var = os.getenv('TEST_VARIABLE')
print(f"TEST_VARIABLE: {test_var}")

TEST_VARIABLE: None
